## Подготовка к работе с Википедией

Скачайте 'enwiki-20200401-pages-articles.xml.bz2' по ссылке https://meta.wikimedia.org/wiki/Data_dump_torrents — архив весит порядка 16Гб

Скачайте 'wiki.corpus' по ссылке https://yadi.sk/d/TVo-KPUbgx4vPA — это слепок памяти объекта для работы с нелемматизированной(!) Википедией

In [2]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus

if os.path.isfile("wiki.corpus"):
    print("Using preloaded object")
    wiki = WikiCorpus.load("wiki.corpus")
else:
    print("Now wait for about 15 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    wiki = WikiCorpus('enwiki-20200401-pages-articles.xml.bz2', lemmatize=False)
    wiki.save('wiki.corpus')
    
for text in wiki.get_texts():
    print(text, type(text))
    break

Using preloaded object


## Сначала рассчитаем корреляцию Спирмена с Жаккаром по окну радиуса 2

In [14]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={i:set() for i in simlex_words}

counter=0
k=2#радиус окна

for text in wiki.get_texts():
    counter+=1
  
    for ind, word in enumerate(text):
        if word in simlex_words:
            l = ind - k if ind - k > 0 else 0
            r = ind + k + 1
            pseudodocs[word].update(text[l:r])
            
    if counter % 10000 == 0:
        simlex_pairs=[]
        jaccard_pairs=[]
        for i in f[1:]:
            ii=i.split('\t')
            w1=ii[0]
            w2=ii[1]
            try:
                jaccard_pairs.append(\
                                    len(pseudodocs[w1].intersection(pseudodocs[w2]))*1./ \
                                    len(pseudodocs[w1].union(pseudodocs[w2]))
                                    )
                simlex_pairs.append(float(ii[3]))
            except:
                print(pseudodocs[w1], pseudodocs[w2])

        coef = spearmanr(simlex_pairs, jaccard_pairs)
        print(counter, coef)
    
    if counter == 200000:
        break

10000 SpearmanrResult(correlation=0.11064909248070758, pvalue=0.0004588995563067517)
20000 SpearmanrResult(correlation=0.1084351638660099, pvalue=0.0005966280197246219)
30000 SpearmanrResult(correlation=0.10610753479225185, pvalue=0.0007821809303677304)
40000 SpearmanrResult(correlation=0.10009823418856723, pvalue=0.0015359764071859177)
50000 SpearmanrResult(correlation=0.09605745331446396, pvalue=0.002371178665128621)
60000 SpearmanrResult(correlation=0.09712560476989328, pvalue=0.0021172657885964903)
70000 SpearmanrResult(correlation=0.09752549212977182, pvalue=0.0020287959415915898)
80000 SpearmanrResult(correlation=0.09722981797062427, pvalue=0.0020938758309202955)
90000 SpearmanrResult(correlation=0.09331948912684855, pvalue=0.003154191743651448)
100000 SpearmanrResult(correlation=0.08980522971504232, pvalue=0.00450205362115943)
110000 SpearmanrResult(correlation=0.08784360372103074, pvalue=0.005463309353402319)
120000 SpearmanrResult(correlation=0.08511109500618216, pvalue=0.0071

После обработки первых 200000 статей можно остановиться и сделать вывод, что корреляция вряд ли станет больше 0.12

Но что интересно, и меньше 0.08 она тоже не стала, что означает, что числитель и знаменатель в коэффициенте Жаккара растут достаточно равномерно

Поэтому имеет смысл попробовать окна другого радиуса

## Затем произведём одновременный расчёт корреляции Спирмена с Жаккаром по нескольким окнам

In [5]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={k:{i:set() for i in simlex_words} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1
    
    for ind, word in enumerate(text):
        if word in simlex_words:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][word].update(text[l:r])
                
    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            simlex_pairs=[]
            jaccard_pairs=[]
            for i in f[1:]:
                ii=i.split('\t')
                first_word=ii[0]
                second_word=ii[1]
                try:
                    jaccard_pairs.append(\
                                        len(pseudodocs[k][first_word].intersection(pseudodocs[k][second_word]))*1./ \
                                        len(pseudodocs[k][first_word].union(pseudodocs[k][second_word]))
                                        )
                    simlex_pairs.append(float(ii[3]))
                except:
                    print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])

            coef = spearmanr(simlex_pairs, jaccard_pairs)
            print(k, counter, coef)
            
    if counter == 200000:
        break

1 10000 SpearmanrResult(correlation=0.16485923876088465, pvalue=1.6051592219945796e-07)
2 10000 SpearmanrResult(correlation=0.11076630186537834, pvalue=0.0004525067637921213)
3 10000 SpearmanrResult(correlation=0.08899130575289914, pvalue=0.004880613957089424)
4 10000 SpearmanrResult(correlation=0.07452602070841902, pvalue=0.018479444921519598)
5 10000 SpearmanrResult(correlation=0.06691854488336273, pvalue=0.03444574341758788)
1 20000 SpearmanrResult(correlation=0.16696755787891576, pvalue=1.1081451599619917e-07)
2 20000 SpearmanrResult(correlation=0.10843447177938247, pvalue=0.0005966765274165116)
3 20000 SpearmanrResult(correlation=0.08702283110789909, pvalue=0.005917711626231574)
4 20000 SpearmanrResult(correlation=0.07570368187059003, pvalue=0.016702015884979885)
5 20000 SpearmanrResult(correlation=0.06885707952098888, pvalue=0.029538693081851194)
1 30000 SpearmanrResult(correlation=0.16286050964575358, pvalue=2.2709115871863328e-07)
2 30000 SpearmanrResult(correlation=0.106107534

1 200000 SpearmanrResult(correlation=0.11080410684966215, pvalue=0.0004504625820866351)
2 200000 SpearmanrResult(correlation=0.08467571233738665, pvalue=0.007410708344663224)
3 200000 SpearmanrResult(correlation=0.07524560477862735, pvalue=0.017374742770183336)
4 200000 SpearmanrResult(correlation=0.0715996442910879, pvalue=0.023628355981486077)
5 200000 SpearmanrResult(correlation=0.07045699443926792, pvalue=0.025953196950864087)


После обработки первых 200000 статей можно сделать вывод, что наилучший результат показало окно с радиусом 1

## Наконец произведём раздельный расчёт по частям речи по нескольким окнам

In [7]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words={w: set() for w in ['A', 'N', 'V']}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words[ii[2]].add(ii[0])
    simlex_words[ii[2]].add(ii[1])

pseudodocs={k:{w:{i:set() for i in simlex_words[w]} for w in ['A', 'N', 'V']} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1

    for ind, word in enumerate(text):
        ww=None
        if word in simlex_words['A']:
            ww='A'
        elif word in simlex_words['N']:
            ww='N'
        elif word in simlex_words['V']:
            ww='V'
        if ww:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][ww][word].update(text[l:r])

    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for w in ['A', 'N', 'V']:
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        if ii[2] == w:
                            jaccard_pairs.append(\
                                                len(pseudodocs[k][w][first_word].intersection(pseudodocs[k][w][second_word]))*1./ \
                                                len(pseudodocs[k][w][first_word].union(pseudodocs[k][w][second_word]))
                            )
                            simlex_pairs.append(float(ii[3]))

                    except:
                        print(k, w, first_word, second_word)#, pseudodocs[k][w][first_word], pseudodocs[k][w][second_word])

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, w, counter, coef)
            
    if counter == 200000:
        break

1 A 10000 SpearmanrResult(correlation=0.17291892033620188, pvalue=0.06954110078372773)
1 N 10000 SpearmanrResult(correlation=0.18481679083251246, pvalue=1.57064978131956e-06)
1 V 10000 SpearmanrResult(correlation=0.09450634729802541, pvalue=0.16052547565075057)
2 A 10000 SpearmanrResult(correlation=0.13670448986759207, pvalue=0.152518135611169)
2 N 10000 SpearmanrResult(correlation=0.12362766199507362, pvalue=0.001389931291825453)
2 V 10000 SpearmanrResult(correlation=0.07415337141694049, pvalue=0.27127180418231567)
3 A 10000 SpearmanrResult(correlation=0.11982650652037191, pvalue=0.21032272798104482)
3 N 10000 SpearmanrResult(correlation=0.10164166383149141, pvalue=0.008666487247528852)
3 V 10000 SpearmanrResult(correlation=0.05892213193284733, pvalue=0.3822689610629747)
4 A 10000 SpearmanrResult(correlation=0.09806739246147103, pvalue=0.3058452580187223)
4 N 10000 SpearmanrResult(correlation=0.09188867804800506, pvalue=0.01769524336270219)
4 V 10000 SpearmanrResult(correlation=0.0427

2 V 70000 SpearmanrResult(correlation=0.07093963637294864, pvalue=0.29264530659766386)
3 A 70000 SpearmanrResult(correlation=0.1047412405122047, pvalue=0.2739382977003108)
3 N 70000 SpearmanrResult(correlation=0.10155958526248934, pvalue=0.008720872023690861)
3 V 70000 SpearmanrResult(correlation=0.050913021594544076, pvalue=0.4503724820770486)
4 A 70000 SpearmanrResult(correlation=0.09116537866541313, pvalue=0.34130111876117275)
4 N 70000 SpearmanrResult(correlation=0.09418949442441157, pvalue=0.015032812550175753)
4 V 70000 SpearmanrResult(correlation=0.03811731066512119, pvalue=0.5721184180585231)
5 A 70000 SpearmanrResult(correlation=0.08958576965932233, pvalue=0.34976659460157034)
5 N 70000 SpearmanrResult(correlation=0.09036606709727789, pvalue=0.019676092679479576)
5 V 70000 SpearmanrResult(correlation=0.0332067673914176, pvalue=0.6226399341584059)
1 A 80000 SpearmanrResult(correlation=0.17770162538242124, pvalue=0.06205692456356705)
1 N 80000 SpearmanrResult(correlation=0.16613

4 A 130000 SpearmanrResult(correlation=0.07899361371292452, pvalue=0.4098730038111129)
4 N 130000 SpearmanrResult(correlation=0.09407445037223941, pvalue=0.015157061176552574)
4 V 130000 SpearmanrResult(correlation=0.02754269648111931, pvalue=0.6831725900114339)
5 A 130000 SpearmanrResult(correlation=0.07712002191958903, pvalue=0.4211017296089018)
5 N 130000 SpearmanrResult(correlation=0.0902052410370556, pvalue=0.01989619585854566)
5 V 130000 SpearmanrResult(correlation=0.022405656181277715, pvalue=0.7398939076077697)
1 A 140000 SpearmanrResult(correlation=0.1280668001688123, pvalue=0.1804025241452368)
1 N 140000 SpearmanrResult(correlation=0.14091522464609002, pvalue=0.00026430780392394947)
1 V 140000 SpearmanrResult(correlation=0.08112706677861292, pvalue=0.22861750227785385)
2 A 140000 SpearmanrResult(correlation=0.08588685190339301, pvalue=0.37009867331104085)
2 N 140000 SpearmanrResult(correlation=0.10619110362346286, pvalue=0.006086530665786748)
2 V 140000 SpearmanrResult(correl

5 N 190000 SpearmanrResult(correlation=0.09593071377760887, pvalue=0.01325864744112122)
5 V 190000 SpearmanrResult(correlation=0.021364208596202538, pvalue=0.7515784966021003)
1 A 200000 SpearmanrResult(correlation=0.12738230293283961, pvalue=0.1827622536708083)
1 N 200000 SpearmanrResult(correlation=0.1381978660473362, pvalue=0.0003476314701701081)
1 V 200000 SpearmanrResult(correlation=0.07266715606383162, pvalue=0.28102042255295634)
2 A 200000 SpearmanrResult(correlation=0.09299509243080165, pvalue=0.3316584406759212)
2 N 200000 SpearmanrResult(correlation=0.10999123371757369, pvalue=0.004485460087239874)
2 V 200000 SpearmanrResult(correlation=0.05322460746663037, pvalue=0.4300468680720049)
3 A 200000 SpearmanrResult(correlation=0.07862503827817001, pvalue=0.4120679534223133)
3 N 200000 SpearmanrResult(correlation=0.10330242953167779, pvalue=0.00762914249506186)
3 V 200000 SpearmanrResult(correlation=0.03773506265732899, pvalue=0.5759808673252795)
4 A 200000 SpearmanrResult(correlat

После обработки первых 200000 статей можно сделать вывод, что лучший результат показали существительные

## Теперь поработаем с лемматизированной Вики

Скачайте 'wiki.lem.corpus' по ссылке https://yadi.sk/d/zGVfIvRNdE2w2A — это слепок памяти объекта для работы с лемматизированной(!) Википедией, но не всей(!!!), а немногим больше первых 400000 статей(лемматизация вычислительно затратный процесс)

Раздельный расчёт по нескольким частям речи в этом разделе не проведён, но далее есть раздел, где проведён расчёт, когда в псевдодокумент к существительным попадают только существительные и т.д. — такая возможность как раз является следствием лемматизации

Т.е. на нелемматизированной Википедии в расчёте по частям речи к существительным могло попасть что угодно, а в лемматизированной можно сделать так, что попадут только существительные

При лемматизации важно не забывать обрезать у слова часть речи — это не должно влиять на показатель качества, но занимает меньше памяти

In [4]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus

if os.path.isfile("wiki.lem.corpus"):
    print("Using preloaded object")
    wiki = WikiCorpus.load("wiki.lem.corpus")
else:
    print("Now wait for about 15 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    wiki = WikiCorpus('enwiki-20200401-pages-articles.xml.bz2', lemmatize=True)
    wiki.save('wiki.lem.corpus')

for text in wiki.get_texts():
    print([word.decode('utf-8').split('/')[0] for word in text]) 
    break

Using preloaded object


In [20]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={k:{i:set() for i in simlex_words} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1
    
    for ind, word in enumerate(text):
        word=word.decode('utf-8').split('/')[0]
        if word in simlex_words:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][word].update([ww.decode('utf-8').split('/')[0] for ww in text[l:r]])
                
    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            simlex_pairs=[]
            jaccard_pairs=[]
            for i in f[1:]:
                ii=i.split('\t')
                first_word=ii[0]
                second_word=ii[1]
                try:
                    jaccard_pairs.append(\
                                        len(pseudodocs[k][first_word].intersection(pseudodocs[k][second_word]))*1./ \
                                        len(pseudodocs[k][first_word].union(pseudodocs[k][second_word]))
                                        )
                    simlex_pairs.append(float(ii[3]))
                except:
                    print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])

            coef = spearmanr(simlex_pairs, jaccard_pairs)
            print(k, counter, coef)
            
    if counter == 200000:
        break

C:\Users\cheptil\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\utils.py:1268: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)


1 10000 SpearmanrResult(correlation=0.07415098970047042, pvalue=0.019079158785807793)
2 10000 SpearmanrResult(correlation=0.05891852908089593, pvalue=0.06267056860938328)
3 10000 SpearmanrResult(correlation=0.058309790168474374, pvalue=0.06543778293280282)
4 10000 SpearmanrResult(correlation=0.058839781606852805, pvalue=0.06302300601484578)
5 10000 SpearmanrResult(correlation=0.05866003437791815, pvalue=0.06383360294441344)
1 20000 SpearmanrResult(correlation=0.07405834888751883, pvalue=0.01922989228829983)
2 20000 SpearmanrResult(correlation=0.06740232644391354, pvalue=0.033160197075306014)
3 20000 SpearmanrResult(correlation=0.06759629674935391, pvalue=0.03265639859597883)
4 20000 SpearmanrResult(correlation=0.06847194323693834, pvalue=0.03046276127600219)
5 20000 SpearmanrResult(correlation=0.06934319045865912, pvalue=0.028406807049993554)
1 30000 SpearmanrResult(correlation=0.07629488833879643, pvalue=0.015867600342628525)
2 30000 SpearmanrResult(correlation=0.06913774895553083, pv

MemoryError: 

Произошло странное — лемматизация ухудшила качество, интерпретация такая, что числитель растёт медленнее знаменателя

Что интересно — ширина окна почти не влияет на качество, но возможно это просто эффект близости к нулю

In [14]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words={w: set() for w in ['A', 'N', 'V']}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words[ii[2]].add(ii[0])
    simlex_words[ii[2]].add(ii[1])

pseudodocs={k:{w:{i:set() for i in simlex_words[w]} for w in ['A', 'N', 'V']} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1

    for ind, word in enumerate(text):
        word=word.decode('utf-8').split('/')[0]
        ww=None
        if word in simlex_words['A']:
            ww='A'
        elif word in simlex_words['N']:
            ww='N'
        elif word in simlex_words['V']:
            ww='V'
        if ww:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][ww][word].update([ww.decode('utf-8').split('/')[0] for ww in text[l:r]])

    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for w in ['A', 'N', 'V']:
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        if ii[2] == w:
                            jaccard_pairs.append(\
                                                len(pseudodocs[k][w][first_word].intersection(pseudodocs[k][w][second_word]))*1./ \
                                                len(pseudodocs[k][w][first_word].union(pseudodocs[k][w][second_word]))
                            )
                            simlex_pairs.append(float(ii[3]))

                    except:
                        print(k, w, first_word, second_word)#, pseudodocs[k][w][first_word], pseudodocs[k][w][second_word])

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, w, counter, coef)
            
    if counter == 200000:
        break
        

Расчёт по нескольким частям речи не был проведен ввиду своей вычислительной трудности

Как можно заметить, эксперименты становятся всё более вычислительно затратными, поэтому учитывая что в предыдущих расчётах показатель качества не сильно менялся, в дальнейших расчётах ограничимся 10000 документами

## Теперь посчитаем tfidf по нелемматизированной Вики и добавим в окно фильтрацию по tfidf

Расчёт по нелемматизированной версии связан с тем, что нет полного объекта для лемматизированной версии

Скачайте 'wiki.tfidf.model' по ссылке https://yadi.sk/d/_lwvMkGZ_l9_BA

In [1]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models import TfidfModel

wiki = WikiCorpus.load("wiki.corpus")

if os.path.isfile("wiki.tfidf.model"):
    print("Using preloaded object")
    tfidf = TfidfModel.load("wiki.tfidf.model")
else:
    print("Now wait for about 9 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    tfidf = TfidfModel(wiki)
    tfidf.save('wiki.tfidf.model')

Using preloaded object


In [38]:
#Генераторы используются для синхронного доступа к документам

def next_text():
    for text in wiki.get_texts():
        yield text

n_t=next_text()

def tfidf_for_text():
    for doc in tfidf[wiki]:
        yield doc
        
t_t=tfidf_for_text()

In [ ]:
#Тест

cur_text = next(n_t)
cur_tfidf = next(t_t)
cur_dir={i:j for i,j in cur_tfidf}

for w in cur_text:
    cur_w_id=wiki.dictionary.token2id[w]
    cur_w_tfidf=cur_dir[cur_w_id]
    print(w, cur_w_tfidf)

In [39]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={k:{t:{i:set() for i in simlex_words} for t in range(1, 11)} for k in [1, 2, 3, 4, 5]}

counter=0
#Раскоментируйте, если хотите каждый раз начинать обработку с первого документа(требуется незначительное время на инициализацию)
#n_t=next_text()
#t_t=tfidf_for_text()

while counter < 10000:
    counter+=1

    cur_text = next(n_t)
    cur_tfidf = next(t_t)
    cur_dir={i:j for i,j in cur_tfidf}
    
    for ind, word in enumerate(cur_text):
        if word in simlex_words:
            #cur_w_id=wiki.dictionary.token2id[word]
            #cur_w_tfidf=cur_dir[cur_w_id]
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                for t in range(1, 11):
                    try:
                        good_w = [item for item in cur_text[l:r] if cur_dir[wiki.dictionary.token2id[item]] > t/1000] #mitimco
                        #print(word, good_w)
                        pseudodocs[k][t][word].update(good_w)
                    except:
                        pass
                
    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for t in range(1, 11):
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        jaccard_pairs.append(\
                                            len(pseudodocs[k][t][first_word].intersection(pseudodocs[k][t][second_word]))*1./ \
                                            len(pseudodocs[k][t][first_word].union(pseudodocs[k][t][second_word]))
                                            )
                        simlex_pairs.append(float(ii[3]))
                    except:
                        #print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])
                        pass

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, t, counter, coef)

1 1 1000 SpearmanrResult(correlation=0.09523837322639907, pvalue=0.0025844162504016716)
1 2 1000 SpearmanrResult(correlation=0.09260775674986609, pvalue=0.0033930807395115004)
1 3 1000 SpearmanrResult(correlation=0.09036420498077022, pvalue=0.004257674611503342)
1 4 1000 SpearmanrResult(correlation=0.0810693316092976, pvalue=0.010366227575204098)
1 5 1000 SpearmanrResult(correlation=0.08523920588554723, pvalue=0.007024371556388656)
1 6 1000 SpearmanrResult(correlation=0.08061778132206862, pvalue=0.010801901498093767)
1 7 1000 SpearmanrResult(correlation=0.07902987143799914, pvalue=0.012465831045554826)
1 8 1000 SpearmanrResult(correlation=0.07353163555748755, pvalue=0.02010679943041202)
1 9 1000 SpearmanrResult(correlation=0.07654945898986988, pvalue=0.015519728778482783)
1 10 1000 SpearmanrResult(correlation=0.07697167491535863, pvalue=0.014957530575768428)
2 1 1000 SpearmanrResult(correlation=0.12333435510103416, pvalue=9.295785120126371e-05)
2 2 1000 SpearmanrResult(correlation=0.12

5 5 2000 SpearmanrResult(correlation=0.10662387502437706, pvalue=0.0007369135967719191)
5 6 2000 SpearmanrResult(correlation=0.11082635323901237, pvalue=0.0004492637059985628)
5 7 2000 SpearmanrResult(correlation=0.11264873711273321, pvalue=0.0003605586890657922)
5 8 2000 SpearmanrResult(correlation=0.10719843128727836, pvalue=0.0006894052037117411)
5 9 2000 SpearmanrResult(correlation=0.10657231681336451, pvalue=0.0007413219996125509)
5 10 2000 SpearmanrResult(correlation=0.10953790138510046, pvalue=0.00052382706579223)
1 1 3000 SpearmanrResult(correlation=0.13836462251178985, pvalue=1.1390626418270668e-05)
1 2 3000 SpearmanrResult(correlation=0.1402800512274067, pvalue=8.575131777829324e-06)
1 3 3000 SpearmanrResult(correlation=0.13886174104670923, pvalue=1.0585246582470997e-05)
1 4 3000 SpearmanrResult(correlation=0.13228908524104188, pvalue=2.735121592796002e-05)
1 5 3000 SpearmanrResult(correlation=0.13476615644286574, pvalue=1.9223320464411188e-05)
1 6 3000 SpearmanrResult(correl

4 9 4000 SpearmanrResult(correlation=0.11135405159512163, pvalue=0.0004216826823626063)
4 10 4000 SpearmanrResult(correlation=0.11372640624493544, pvalue=0.00031609806260822867)
5 1 4000 SpearmanrResult(correlation=0.06187423258268628, pvalue=0.050573449862980985)
5 2 4000 SpearmanrResult(correlation=0.06479994474114877, pvalue=0.04058768134433761)
5 3 4000 SpearmanrResult(correlation=0.07087567782447819, pvalue=0.025079378817801787)
5 4 4000 SpearmanrResult(correlation=0.08056194739684021, pvalue=0.010856885983445934)
5 5 4000 SpearmanrResult(correlation=0.08493089120541818, pvalue=0.007233457359631252)
5 6 4000 SpearmanrResult(correlation=0.09026266094287286, pvalue=0.004301155546986344)
5 7 4000 SpearmanrResult(correlation=0.09285907181489352, pvalue=0.0033069113559226803)
5 8 4000 SpearmanrResult(correlation=0.0924673964739625, pvalue=0.003442090194957268)
5 9 4000 SpearmanrResult(correlation=0.09761924366707228, pvalue=0.0020085508628783608)
5 10 4000 SpearmanrResult(correlation=0

4 3 6000 SpearmanrResult(correlation=0.08526095608970231, pvalue=0.0070098281905281645)
4 4 6000 SpearmanrResult(correlation=0.09300062158549059, pvalue=0.0032592584528670495)
4 5 6000 SpearmanrResult(correlation=0.09882322766213533, pvalue=0.0017645035762781753)
4 6 6000 SpearmanrResult(correlation=0.10514653402777963, pvalue=0.0008733635487622744)
4 7 6000 SpearmanrResult(correlation=0.1070998779332004, pvalue=0.0006973468924906074)
4 8 6000 SpearmanrResult(correlation=0.10957395692033636, pvalue=0.0005215925480153878)
4 9 6000 SpearmanrResult(correlation=0.11540809750736035, pvalue=0.0002568288106942497)
4 10 6000 SpearmanrResult(correlation=0.11432331119923504, pvalue=0.00029373245678489544)
5 1 6000 SpearmanrResult(correlation=0.06890287521407215, pvalue=0.029430432398691034)
5 2 6000 SpearmanrResult(correlation=0.07066347375832831, pvalue=0.025519030669001266)
5 3 6000 SpearmanrResult(correlation=0.07417710472042809, pvalue=0.019036854490899643)
5 4 6000 SpearmanrResult(correlati

3 7 8000 SpearmanrResult(correlation=0.11650829489480265, pvalue=0.00022386871194188447)
3 8 8000 SpearmanrResult(correlation=0.11807764063851768, pvalue=0.00018365799908547727)
3 9 8000 SpearmanrResult(correlation=0.12029238002426282, pvalue=0.00013831428190311677)
3 10 8000 SpearmanrResult(correlation=0.12061610770271272, pvalue=0.00013264491043825873)
4 1 8000 SpearmanrResult(correlation=0.07684851351182256, pvalue=0.01511964472132903)
4 2 8000 SpearmanrResult(correlation=0.07859161563218842, pvalue=0.012963284855208083)
4 3 8000 SpearmanrResult(correlation=0.08167151800514787, pvalue=0.009809535804407039)
4 4 8000 SpearmanrResult(correlation=0.08806695536868706, pvalue=0.0053452243024217)
4 5 8000 SpearmanrResult(correlation=0.09177222744354892, pvalue=0.0036944383788456345)
4 6 8000 SpearmanrResult(correlation=0.09817598252152737, pvalue=0.001892092410473249)
4 7 8000 SpearmanrResult(correlation=0.10453815471523432, pvalue=0.0009360742526343537)
4 8 8000 SpearmanrResult(correlatio

KeyboardInterrupt: 

Фильтрация по tfidf положительно влияет на показатель качества - наилучший результат показал порог 0.008 на окне радиуса 1, что согласуется с предыдущими расчётами

Подобный расчёт также необходимо сделать на лемматизированной версии Википедии(если расчёт на лемматизированной версии без фильтрации по tfidf верен, качество должно ухудшиться)

Также этот расчёт стоит сделать раздельно по частям речи(три вектора: для N, A, V, но в псевдодокументах слов могут быть любые части речи)

## Теперь проведём другую фильтрацию - по частям речи на лемматизированной Вики

Фильтрация заключается в том, что в окне заданного радиуса остаются только слова, относящиеся к той же части речи, что и центральное слово

In [ ]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus

if os.path.isfile("wiki.lem.corpus"):
    print("Using preloaded object")
    wiki = WikiCorpus.load("wiki.lem.corpus")
else:
    print("Now wait for about 15 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    wiki = WikiCorpus('enwiki-20200401-pages-articles.xml.bz2', lemmatize=True)
    wiki.save('wiki.lem.corpus')

for text in wiki.get_texts():
    print([word.decode('utf-8').split('/') for word in text]) 
    break

In [8]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
simlex_pos={}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])
    if ii[2] == 'A':
        simlex_pos[ii[0]]='JJ'
        simlex_pos[ii[1]]='JJ'
    if ii[2] == 'N':
        simlex_pos[ii[0]]='NN'
        simlex_pos[ii[1]]='NN'
    if ii[2] == 'V':
        simlex_pos[ii[0]]='VB'
        simlex_pos[ii[1]]='VB'
    
pseudodocs={k:{i:set() for i in simlex_words} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1
    
    for ind, word in enumerate(text):
        word_pos=word.decode('utf-8').split('/')[1]
        word=word.decode('utf-8').split('/')[0]
        if word in simlex_words:
            if simlex_pos[word] != word_pos:
                #print(word, simlex_pos[word], word_pos)
                pass
            else:
                for k in [1, 2, 3, 4, 5]:
                    l = ind - k if ind - k > 0 else 0
                    r = ind + k + 1
                    good_w = [item.decode('utf-8').split('/')[0] for item in text[l:r] if item.decode('utf-8').split('/')[1] == word_pos]
                    #print(word, good_w)
                    pseudodocs[k][word].update(good_w)
                
    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            simlex_pairs=[]
            jaccard_pairs=[]
            for i in f[1:]:
                ii=i.split('\t')
                first_word=ii[0]
                second_word=ii[1]
                try:
                    jaccard_pairs.append(\
                                        len(pseudodocs[k][first_word].intersection(pseudodocs[k][second_word]))*1./ \
                                        len(pseudodocs[k][first_word].union(pseudodocs[k][second_word]))
                                        )
                    simlex_pairs.append(float(ii[3]))
                except:
                    #print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])
                    pass

            coef = spearmanr(simlex_pairs, jaccard_pairs)
            print(k, counter, coef)
            
    if counter == 10000:
        break        

1 1000 SpearmanrResult(correlation=0.12645279026569475, pvalue=6.126542750037106e-05)
2 1000 SpearmanrResult(correlation=0.0843109534850985, pvalue=0.007670830481299786)
3 1000 SpearmanrResult(correlation=0.07121483019740889, pvalue=0.024390305322868833)
4 1000 SpearmanrResult(correlation=0.05946490551918803, pvalue=0.06026980738303623)
5 1000 SpearmanrResult(correlation=0.050843057904007845, pvalue=0.10826920424415915)
1 2000 SpearmanrResult(correlation=0.1162131611794146, pvalue=0.00023229814651605038)
2 2000 SpearmanrResult(correlation=0.06591612832088169, pvalue=0.037245216634192835)
3 2000 SpearmanrResult(correlation=0.06928541978681978, pvalue=0.028539339754025707)
4 2000 SpearmanrResult(correlation=0.058721459466873946, pvalue=0.06355563567710754)
5 2000 SpearmanrResult(correlation=0.05924838791726884, pvalue=0.06121191290508349)
1 3000 SpearmanrResult(correlation=0.1005364008395308, pvalue=0.0014639509233100457)
2 3000 SpearmanrResult(correlation=0.07203293749977042, pvalue=0.0

Фильтрация по частям речи, возможная только при лемматизации, ухудшила качество, по какой-то причине лемматизация не полезна

Что интересно — в случае лемматизации ширина окна влияет на качество, окно радиуса 1 заметно лучше всех остальных окон

In [14]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words={w: set() for w in ['A', 'N', 'V']}
simlex_pos={}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words[ii[2]].add(ii[0])
    simlex_words[ii[2]].add(ii[1])
    if ii[2] == 'A':
        simlex_pos[ii[0]]='JJ'
        simlex_pos[ii[1]]='JJ'
    if ii[2] == 'N':
        simlex_pos[ii[0]]='NN'
        simlex_pos[ii[1]]='NN'
    if ii[2] == 'V':
        simlex_pos[ii[0]]='VB'
        simlex_pos[ii[1]]='VB'

pseudodocs={k:{w:{i:set() for i in simlex_words[w]} for w in ['A', 'N', 'V']} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1

    for ind, word in enumerate(text):
        word_pos=word.decode('utf-8').split('/')[1]
        word=word.decode('utf-8').split('/')[0]
        ww=None
        if word in simlex_words['A']:
            ww='A'
        elif word in simlex_words['N']:
            ww='N'
        elif word in simlex_words['V']:
            ww='V'
        if ww: #if word in simlex_words:
            if simlex_pos[word] != word_pos:
                #print(word, simlex_pos[word], word_pos)
                pass
            else:
                for k in [1, 2, 3, 4, 5]:
                    l = ind - k if ind - k > 0 else 0
                    r = ind + k + 1
                    #pseudodocs[k][ww][word].update([ww.decode('utf-8').split('/')[0] for ww in text[l:r]])
                    good_w = [item.decode('utf-8').split('/')[0] for item in text[l:r] if item.decode('utf-8').split('/')[1] == word_pos]
                    #print(word, good_w)
                    pseudodocs[k][ww][word].update(good_w)                         

    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for w in ['A', 'N', 'V']:
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        if ii[2] == w:
                            jaccard_pairs.append(\
                                                len(pseudodocs[k][w][first_word].intersection(pseudodocs[k][w][second_word]))*1./ \
                                                len(pseudodocs[k][w][first_word].union(pseudodocs[k][w][second_word]))
                            )
                            simlex_pairs.append(float(ii[3]))

                    except:
                        print(k, w, first_word, second_word)#, pseudodocs[k][w][first_word], pseudodocs[k][w][second_word])

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, w, counter, coef)
            
    if counter == 10000:
        break        

Расчёт по нескольким частям речи также не был проведен ввиду своей вычислительной трудности

## Промежуточные результаты

- Нелемматизированная - без фильтрации - все слова вместе - 1 200000 SpearmanrResult(correlation=0.11080410684966215,
- Нелемматизированная - без фильтрации - группировка по частям речи - 1 N 200000 SpearmanrResult(correlation=0.1381978660473362,


- Лемматизированная - без фильтрации - все слова вместе - 1 120000 SpearmanrResult(correlation=0.07523338524627554
- Лемматизированная - без фильтрации - группировка по частям речи - не подсчитано(есть заготовка кода)


- Нелемматизированная - с фильтрацией по tfidf - все слова вместе - 1 8 9000 SpearmanrResult(correlation=0.18086096047200925,
- Нелемматизированная - с фильтрацией по tfidf - группировка по частям речи - не подсчитано


- Лемматизированная - с фильтрацией по частям речи - все слова вместе - 1 10000 SpearmanrResult(correlation=0.104834731509738,
- Лемматизированная - с фильтрацией по частям речи - группировка по частям речи - не подсчитано(есть заготовка кода)


Итого есть 4 параметра: лемматизация, фильтрация по tfidf, фильтрация по частям речи, группировка по частям речи(3 расчёта корреляции Спирмена)

## To Do

Проверить, что при лемматизации числитель растёт медленнее знаменателя по сравнению с нелемматизацией

Проверить, что направления шкал совпадают — т.е. больше значит ближе(пример расчёта качества из Интернета: https://nlp.gluon.ai/examples/word_embedding_evaluation/word_embedding_evaluation.html)

Выяснить, как организовать доступ за O(1) к произвольной статье

Настроить сериализацию промежуточных вычислений